<a href="https://colab.research.google.com/github/Azhar-999-coder/conversational_bot/blob/master/Response_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://nlp.stanford.edu/data/glove.twitter.27B.zip

--2020-07-17 14:21:34--  http://nlp.stanford.edu/data/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.twitter.27B.zip [following]
--2020-07-17 14:21:34--  https://nlp.stanford.edu/data/glove.twitter.27B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip [following]
--2020-07-17 14:21:35--  http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520408563 (1.4G) [appli

In [ ]:
!unzip glove.twitter.27B.zip

Archive:  glove.twitter.27B.zip
  inflating: glove.twitter.27B.25d.txt  
  inflating: glove.twitter.27B.50d.txt  
  inflating: glove.twitter.27B.100d.txt  
  inflating: glove.twitter.27B.200d.txt  


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply, GRU
from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda
from tensorflow.keras.models import Model
import numpy as np
import os
import matplotlib.pyplot as plt
import joblib
import pickle

In [ ]:
BATCH_SIZE = 100
n_a = 64
n_s = 128

In [ ]:
num_words=30000
max_length=30
Tx = max_length
Ty = max_length

path = 'Data/'

dirlist = os.listdir(path)
human_sentences=[]
machine_sentences=[]
for File in dirlist:
    with open(path+"/"+File, 'r') as raw_lines:
        lineList = []
        while True:        
            line = raw_lines.readline()
            if not line:
                break
            lineList.append(line)

    for i in range(0, len(lineList)):
        if(i%2)==0:
            human_sentences.append(lineList[i])
        else:
            machine_sentences.append(lineList[i])    

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=num_words, oov_token='<OOV>')
tokenizer.fit_on_texts(human_sentences)
human_word_index = tokenizer.word_index
human_reverse_word_index = {a:b for (b,a) in human_word_index.items()}
tokenizerE=tokenizer

tokenizer2 = tf.keras.preprocessing.text.Tokenizer(num_words=num_words, oov_token='<OOV>')
    
tokenizer2.fit_on_texts(machine_sentences)
machine_word_index = tokenizer2.word_index
machine_reverse_word_index = {a:b for (b,a) in machine_word_index.items()}

human_sequences = tokenizer.texts_to_sequences(human_sentences)
human_padded = tf.keras.preprocessing.sequence.pad_sequences(human_sequences, maxlen=max_length, padding='post', truncating="post")

machine_sequences = tokenizer2.texts_to_sequences(machine_sentences)
machine_padded = tf.keras.preprocessing.sequence.pad_sequences(machine_sequences, maxlen=max_length, padding="post", truncating="post")


X = human_padded
Y = machine_padded
human_vocab = human_word_index
reverse_human_vocab = human_reverse_word_index
machine_vocab = machine_word_index
reverse_machine_vocab = machine_reverse_word_index


In [ ]:
#taking care of odd-even shit
if X.shape[0]>Y.shape[0]:
  X = np.delete(X, len(X)-1, axis=0)
elif X.shape[0]<Y.shape[0]:
  Y = Y.delete(Y, len(Y)-1, axis=0)

In [ ]:
#creating dataset
dataset = tf.data.Dataset.from_tensor_slices((X,Y))
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
print(X.shape)

(50000, 30)


In [ ]:
EMBEDDING_DIM = 200
embeddings_index = {}
f = open('glove.twitter.27B.200d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

embedding_matrix = np.zeros((len(lda_model.id2word) + 1, EMBEDDING_DIM)) #lda_model.id2word dicitonary
for  i,word in lda_model.id2word.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

from tensorflow.keras.layers import Embedding

embedding_layer_keywords = Embedding(len(lda_model.id2word) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            #input_length=MAX_SEQ_LENGTH,
                            #mask_zero = True,
                            trainable=False)

Found 1193514 word vectors.


In [ ]:
word_to_index_keywords = {word:index for index,word in lda_model.id2word.items()}
keywords_seq_master=[]
c=0
for i in human_sentences:
  c=c+1
  if(((c/len(human_sentences))*100)%10==0):
    print((c/len(human_sentences))*100, "% complete")
  keywords = get_keywords(lda_model,i,3,5)
  keywords_seq = [word_to_index_keywords[key] for key in keywords]
  keywords_seq_master.append(keywords_seq)

10.0 % complete
20.0 % complete
30.0 % complete
40.0 % complete
50.0 % complete
60.0 % complete
70.0 % complete
80.0 % complete
90.0 % complete
100.0 % complete


In [ ]:
K_master = embedding_layer_keywords(np.array(keywords_seq_master))
print(K_master.shape)

(50000, 15, 200)


In [ ]:
#creating dataset
Kdataset = tf.data.Dataset.from_tensor_slices((K_master))
Kdataset = Kdataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, batch_size=100, dim=max_length, shuffle=False):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.len_per_epoch = int(len(X)/self.batch_size)
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return self.len_per_epoch

    def __getitem__(self, index):
        
        p, q = self.__data_generation()

        return p, q

    def __data_generation(self):
        
        X = np.empty((self.batch_size, self.dim))
        y = np.empty((self.batch_size, self.dim))
        s0 = np.zeros((self.batch_size, n_s))
        c0 = np.zeros((self.batch_size, n_s))
        X, y = next(iter(dataset))
        y = tf.transpose(y)
        K = next(iter(Kdataset))

        return [X,s0,c0,K], y

In [ ]:
embedding_dim=200
Embedding_matrix = np.zeros((len(human_vocab)+1, embedding_dim))
for word, i in human_vocab.items():
    Embedding_vector = embeddings_index.get(word)
    if Embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        Embedding_matrix[i] = Embedding_vector

In [ ]:
embeddingLayer = tf.keras.layers.Embedding((len(human_vocab)+1),embedding_dim, weights=[Embedding_matrix], trainable=False, input_length=max_length)

In [ ]:
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor = Dense(1, activation = "relu")
dotor = Dot(axes = 1)

In [ ]:
def one_step_attention(a, s_prev):
    
    s_prev = repeator(s_prev)
    
    concat = concatenator([a, s_prev])
    
    e = densor(concat)
    
    alphas = tf.nn.softmax(e, axis=1)
    
    context = dotor([alphas, a])
    
    return context

In [ ]:
post_activation_LSTM_cell = LSTM(n_s, return_state = True)

In [ ]:
import gensim
import pandas

In [ ]:
!unzip non_mallet_lda_model_saved_through_gensim-20200706T073922Z-001.zip

Archive:  non_mallet_lda_model_saved_through_gensim-20200706T073922Z-001.zip
  inflating: non_mallet_lda_model_saved_through_gensim/lda_non_mallet_model  
  inflating: non_mallet_lda_model_saved_through_gensim/lda_non_mallet_model.id2word  
  inflating: non_mallet_lda_model_saved_through_gensim/lda_non_mallet_model.expElogbeta.npy  
  inflating: non_mallet_lda_model_saved_through_gensim/lda_non_mallet_model.state  


In [ ]:
lda_model = gensim.models.ldamodel.LdaModel.load('model/lda_non_mallet_model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
def get_keywords(lda_model,sentence,take_topics=3,keywords_per_topic = 10): #for eg:returns the list of top 10 keywords from top 3 topics
  text = sentence.split()
  corpus = lda_model.id2word.doc2bow(text)#converting to doc2bow format the original sentence

  prob = lda_model.get_document_topics(corpus) #probability distribution
  #print(prob)
  prob_sorted = sorted(prob,key = lambda x: x[1],reverse=True)#topics sorted in descending order
  #print(prob_sorted)
  all_keywords = []
  for i,topic in enumerate(prob_sorted):
    list = lda_model.print_topic(topic[0],keywords_per_topic).split(" + ")
    keywords = [str.split("*")[1].replace('"','') for str in list]
    all_keywords+=keywords
    if(i==take_topics-1):
      break
  return all_keywords

In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras import Model

In [ ]:
# these two can be tuned and should be changed accordingly as the input to get_keywords function is changed
take_topics = 3
keywords_per_topic = 5

# Defined shared layers as global variables
repeator_topic = RepeatVector(take_topics*keywords_per_topic)  #repeat for total number of topics
concatenator = Concatenate(axis=-1)
dotor = Dot(axes = 1)


#layers for topic attention
densor1_topic = Dense(10, activation = "tanh")
densor2_topic = Dense(1, activation = "softmax")


In [ ]:
def one_step_attention_topic(a_last, s_prev, K): #a_last is last cell state , K is the list of embeddings of keywords
    #calculating context vector
    

    #repeating s_prev and a_last for len(K) times

    s_prev_temp = repeator_topic(s_prev)
    print("a:",a_last.shape,"\n","s:",s_prev_temp.shape,"\n","K:",K.shape)    


    concat = concatenator([a_last,s_prev_temp,K])
    e = densor1(concat)
    alphas = densor2(e)
    topic_vector = dotor([alphas,K])
    
    return topic_vector

In [ ]:
def include_yhat_and_topic(context,out):
  reduced = [tf.math.argmax(out,axis=1)]
  reduced = tf.convert_to_tensor(reduced)
  reduced = tf.transpose(reduced)
  reduced = tf.cast(reduced, tf.dtypes.float32)
  concatable = RepeatVector(1)(reduced)
  concat = Concatenate(axis=-1)([context,concatable])
  feedable = Dense(n_s, activation='relu')(concat)
  return feedable


In [ ]:
def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    
    X = Input(shape=(Tx,))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    K_master = Input(shape=(15,200,))
    
    outputs = []

    x = embeddingLayer(X)
    
    # Define pre-attention Bi-LSTM
    a = Bidirectional(LSTM(n_a, return_sequences=True))(x)
    
    # Iterate for Ty steps
    for t in range(Ty):
    
        # Create context
        context = one_step_attention(a, s)
        print(s.shape)
        def fn(K):
          one_step_attention_topic(a, s, K)
        @tf.function
        def func(tensor):
          return tf.map_fn(fn, tensor)

        
        topic_vector = func(K_master)
        print("topic vector shape:",topic_vector.shape)  

        # if t=0:
        #   context = include_topic(context, topic_vector)
        if t!=0:          
          context = include_yhat_and_topic(context, out)
        # Apply the post-attention LSTM cell to the "context" vector.
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, c])
        
        # Apply Dense layer to the hidden state output of the post-attention LSTM
        p = Dense(machine_vocab_size)(s)

        out = tf.nn.softmax(p,axis=1)
        print("this point was reached")
      
        
        # Append "out" to the "outputs" list and covert it into tf tensor later
        outputs.append(out)
    
    outputs = tf.convert_to_tensor(outputs)

    # Create model instance taking three inputs and returning the tensor of outputs
    model = Model([X, s0, c0], outputs)
    
    return model

In [ ]:
model = model(Tx,Ty,n_a,n_s,len(human_vocab),len(machine_vocab))
model.summary()

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0025)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt)

In [ ]:
generator = DataGenerator()
model.fit_generator(generator=generator, epochs=7)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/7
500/500 [==============================] - 81s 163ms/step - loss: 1.1150
Epoch 2/7
500/500 [==============================] - 81s 163ms/step - loss: 0.4382
Epoch 3/7
500/500 [==============================] - 81s 162ms/step - loss: 0.0369
Epoch 4/7
500/500 [==============================] - 81s 162ms/step - loss: 0.0104
Epoch 5/7
500/500 [==============================] - 81s 162ms/step - loss: 0.0073
Epoch 6/7
500/500 [==============================] - 81s 161ms/step - loss: 0.0064
Epoch 7/7
500/500 [==============================] - 81s 162ms/step - loss: 0.0059


In [ ]:
Examples = ['What are you doing?','Hey','Who are you?','Hope you are having a nice day']
sequenceE = tokenizer.texts_to_sequences(Examples)
X = tf.keras.preprocessing.sequence.pad_sequences(sequences=sequenceE, maxlen=max_length, padding='post', truncating = "post")
print(X)
print(X.shape)
s0 = np.zeros((len(Examples),n_s))
c0 = np.zeros((len(Examples),n_s))
prediction = model.predict([X,s0,c0])
perm = [1,0,2]
prediction = tf.transpose(prediction, perm=perm)
responses = []
prediction = np.array(prediction)

for i in range(0,prediction.shape[0]):
  temp = []
  for j in range(0, prediction.shape[1]):
    p = np.argmax(prediction[i][j])
    temp.append(p)
  responses.append(temp)

final_responses = []
print(responses)
for k in range(0, len(responses)):
  tempo = []
  for l in range(0,len(responses[k])):
    if responses[k][l]!=0:
      kkk = reverse_machine_vocab[responses[k][l]]
      tempo.append(kkk)
  final_responses.append(tempo)
print("\n")
for m in range(0,len(final_responses)):
  print(Examples[m],"\n")
  print("-->",' '.join(final_responses[m]),"\n\n") 


[[ 14  33   2 257   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0]
 [161   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0]
 [ 82  33   2   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0]
 [359   2  33 634   6 226 193   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0]]
(4, 30)
[[382, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [9, 28, 305, 477, 17, 983, 883, 264, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [43, 24, 47, 679, 100, 4730, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [4, 47, 185, 3, 53, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


What are you doing? 

--> under it 


Hey 

--> and don't forget cut your date short tonight